# [Median of Two Sorted Arrays](https://leetcode.com/problems/median-of-two-sorted-arrays/)

There are two sorted arrays nums1 and nums2 of size m and n respectively.

Find the median of the two sorted arrays. The overall run time complexity should be O(log (m+n)).

You may assume nums1 and nums2 cannot be both empty.

# Approach

Apparently the classic approach is to look at the smaller of the two arrays.  Assume that the smaller array is `arr1` and the larger `arr2`, with lengths `N1` and `N2` respectively.  Let `N = N1 + N2` and let `L = (N - 1) // 2`.  We want to find `i` in 

1. `arr1[i]         <= min(arr1[i + 1], arr2[L - i])`
2. `arr2[L - i - 1] <= min(arr1[i + 1], arr2[L - i])`
3. If condition 1 is violated then `i` is too big and you set the upper search range to `i-1`.
4. If condition 2 is violated then `i` is not big enough and you set the lower search range to `i-1`.
5. If you find all valid `i` violate 1, then all of `arr1` is in the upper half of the two arrays, and your left median is at `arr2[L - 1]`.
6. On the other hand, you cannot have all valid `i` violate 2, since then you would be including all of `arr1` in the left half, and still find that `arr2[N2 - 1]` didn't satisfy the condition, which would be a contradiction.

Once you have the left median, `max(arr[i], arr2[L - i - 2])`, then if `N` is odd, you are done.  If `N` is even, then tracking down the right median is simply a matter of computing `min(arr1[i + 1], arr2[L - i])`.

### Solution

Below, this can be much faster.  I think I'm handling the edge cases clumsily and I suspect I can check conditions faster.

In [1]:
def is_left_median_pos(i, arr1, arr2, L):
    '''
    Takes i, arr1, arr2, L, and checks if max(arr1[i], arr2[L - i - 2]) gives
    the left median.
    Returns:
        - 'left median' if the position defines a left median
        - 'i too high' if i is too large
        - 'i too low' if i is too large
    '''
    # i is the last index of arr1
    if i + 1 == len(arr1):
        if arr1[i] <= arr2[L - i]:
            return 'left median'
        else:
            return 'i too high'
    
    else:
        if arr1[i] > arr2[L - i]:
            return 'i too high'
        elif arr2[L - i - 1] > arr1[i + 1]:
            return 'i too low'
        else:
            return 'left median'

def median_sorted_arrays(arr1, arr2):
    N1 = len(arr1)
    N2 = len(arr2)

    
    ### HANDLE EMPTY arr1 ###

    if N1 == 0:
        if N2 % 2 == 1:
            return arr2[N2 // 2]
        else:
            return (arr2[(N2 - 1) // 2] + arr2[N2 // 2]) / 2

        
    ### FIND LEFT MEDIAN ###
    
    N = N1 + N2
    L = (N - 1) // 2
    lower = 0
    upper = N1 - 1
    left_median_pos = -999 
    
    while lower <= upper:
        current = (lower + upper) // 2
        status = is_left_median_pos(current, arr1, arr2, L)
        
        if status == 'left median':
            left_median_pos = current
            break
            
        elif status == 'i too low':
            lower = current + 1
            
        # all of arr1 belongs to the upper half, set left_median_pos = -1 as flag
        elif status == 'i too high' and current == 0:
            left_median_pos = -1
            break
            
        elif status == 'i too high':
            upper = current - 1
                
    ### FIND THE MEDIAN ###
    if N % 2 == 1:
        if left_median_pos == -1:
            return arr2[L]
        
        return max(arr1[left_median_pos], arr2[L - left_median_pos - 1])
    
    elif N % 2 == 0:
        if left_median_pos == -1:
            left_median = arr2[L]
            
            if N1 == N2:
                right_median = arr1[0]
            
            else:
                right_median = min(arr1[0], arr2[L + 1])
            
        elif left_median_pos == N1 - 1:
            if N1 == N2:
                left_median = arr1[left_median_pos]
                
            else:
                left_median = max(arr1[left_median_pos], arr2[L - left_median_pos - 1])
            
            right_median = arr2[L - left_median_pos]
            
        else:
            left_median = max(arr1[left_median_pos], arr2[L - left_median_pos - 1])
            right_median = min(arr1[left_median_pos + 1], arr2[L - left_median_pos])
        
        return (left_median + right_median) / 2

class Solution:
    def findMedianSortedArrays(self, arr1, arr2):
        if len(arr1) > len(arr2):
            return median_sorted_arrays(arr2, arr1)
        
        else:
            return median_sorted_arrays(arr1, arr2)

In [2]:
import unittest

class mytests(unittest.TestCase):
    
    def test_known_results(self):
        x = Solution()

        self.assertEqual(x.findMedianSortedArrays([], [1,2,3,4]), 2.5)
        self.assertEqual(x.findMedianSortedArrays([1,2,11], []), 2)
        self.assertEqual(x.findMedianSortedArrays([1,2,4,100,134], [1,2,3,6,7,7,7,11,99]), 6.5)
        self.assertEqual(x.findMedianSortedArrays([5,6,7,7,7,8], [1,2,3,4,4,4]), 4.5)
        self.assertEqual(x.findMedianSortedArrays([1,2,3,4,4,4], [5,6,7,7,7,8,10,11]), 5.5)
        self.assertEqual(x.findMedianSortedArrays([1,1,3,3], [1,1,3,3]), 2)
        self.assertEqual(x.findMedianSortedArrays([1,4,5], [2,3,6]), 3.5)
        
    
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
